## Key Takeaways

### Widget Types
- **Input**: Button, Slider, FloatSlider, Text, Dropdown, Checkbox, RadioButtons
- **Display**: HTML, Label, Output, Tab
- **Layout**: VBox (vertical), HBox (horizontal), GridBox

### Event Handling
- `.observe()` - Listens to widget value changes
- `.on_click()` - Listens to button clicks
- `interact()` / `interactive()` - Automatic widget generation

### Best Practices
1. Use `Output()` widgets to capture and update results
2. Organize with `VBox()` and `HBox()` for clean layouts
3. Use `clear_output()` to refresh display areas
4. Maintain state in dictionaries for dashboard applications
5. Add `style` and `layout` parameters for visual polish

### Common Patterns
- Real-time updates with sliders
- Tabbed interfaces for different views
- Status messages for user feedback
- CSV export for data persistence

For more information, see [ipywidgets documentation](https://ipywidgets.readthedocs.io/)

In [ ]:
# ============================================
# COMPLETE INTERACTIVE ML DATASET DASHBOARD
# ============================================

# State management
state = {
    'df': None,
    'n_samples': 200,
    'noise': 0.1,
    'test_size': 0.2,
    'random_state': 42
}

# Output areas
output_data = widgets.Output()
output_viz = widgets.Output()
output_metrics = widgets.Output()

# Controls
title_label = widgets.HTML(
    '<h2 style="text-align: center; color: #2C3E50;">🔧 ML Dataset Generator Dashboard</h2>'
)

# Dataset Section
dataset_header = widgets.HTML('<h4>Dataset Configuration</h4>')
n_samples_slider = widgets.IntSlider(
    value=200, min=50, max=1000, step=50, description='Samples:'
)
noise_slider = widgets.FloatSlider(
    value=0.1, min=0, max=1, step=0.05, description='Noise Level:'
)

# Model Section
model_header = widgets.HTML('<h4>Model Configuration</h4>')
test_size_slider = widgets.FloatSlider(
    value=0.2, min=0.1, max=0.5, step=0.05, description='Test Size:'
)
model_type = widgets.RadioButtons(
    options=['Classification', 'Regression'],
    description='Model Type:'
)

# Action Buttons
generate_btn_dash = widgets.Button(
    description='Generate Dataset',
    button_style='success',
    icon='magic',
    layout=widgets.Layout(width='180px')
)

clear_btn = widgets.Button(
    description='Clear Dashboard',
    button_style='warning',
    icon='trash',
    layout=widgets.Layout(width='180px')
)

export_btn = widgets.Button(
    description='Export Data (CSV)',
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='180px')
)

# Status display
status_output = widgets.HTML('<p style="color: #7F8C8D;"><i>Ready...</i></p>')

# Function to generate classification or regression data
def generate_dashboard_data():
    from sklearn.datasets import make_classification, make_regression
    
    n_samples = n_samples_slider.value
    noise = noise_slider.value
    test_size = test_size_slider.value
    
    if model_type.value == 'Classification':
        X, y = make_classification(
            n_samples=n_samples,
            n_features=4,
            n_informative=3,
            n_redundant=1,
            random_state=state['random_state']
        )
        y = y.astype(int)
    else:
        X, y = make_regression(
            n_samples=n_samples,
            n_features=4,
            n_informative=3,
            random_state=state['random_state']
        )
    
    # Add noise
    X = X + np.random.normal(0, noise, X.shape)
    
    # Create DataFrame
    feature_names = [f'Feature {i+1}' for i in range(X.shape[1])]
    df = pd.DataFrame(X, columns=feature_names)
    df['Target'] = y
    
    state['df'] = df
    
    # Update status
    status_output.value = f'<p style="color: #27AE60;"><b>✓ Generated {n_samples} samples</b></p>'
    
    # Update data tab
    with output_data:
        clear_output()
        display(df.head(10))
        display(df.describe().round(3))
    
    # Update visualization tab
    with output_viz:
        clear_output()
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        
        # Histograms of features
        for idx, col in enumerate(feature_names):
            ax = axes[idx // 2, idx % 2]
            ax.hist(df[col], bins=20, alpha=0.7, color='steelblue', edgecolor='black')
            ax.set_title(f'Distribution of {col}')
            ax.set_xlabel(col)
            ax.set_ylabel('Frequency')
        
        plt.tight_layout()
        plt.show()
    
    # Update metrics tab
    with output_metrics:
        clear_output()
        
        metrics_data = {
            'Metric': [
                'Total Samples',
                'Training Samples',
                'Test Samples',
                'Features',
                'Data Type'
            ],
            'Value': [
                n_samples,
                int(n_samples * (1 - test_size)),
                int(n_samples * test_size),
                len(feature_names),
                model_type.value
            ]
        }
        metrics_df = pd.DataFrame(metrics_data)
        display(metrics_df)
        
        print('\nCorrelation Matrix:')
        corr_matrix = df.corr()
        print(corr_matrix.round(3))

def on_generate_dash(b):
    try:
        generate_dashboard_data()
    except Exception as e:
        status_output.value = f'<p style="color: #E74C3C;"><b>✗ Error: {str(e)}</b></p>'

def on_clear_dash(b):
    state['df'] = None
    with output_data:
        clear_output()
        display(widgets.HTML('<p style="color: #95A5A6;"><i>No data loaded</i></p>'))
    with output_viz:
        clear_output()
    with output_metrics:
        clear_output()
    status_output.value = '<p style="color: #7F8C8D;"><i>Cleared...</i></p>'

def on_export_dash(b):
    if state['df'] is None:
        status_output.value = '<p style="color: #E74C3C;"><b>✗ No data to export</b></p>'
    else:
        filename = f"dataset_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        state['df'].to_csv(filename, index=False)
        status_output.value = f'<p style="color: #27AE60;"><b>✓ Exported to {filename}</b></p>'

generate_btn_dash.on_click(on_generate_dash)
clear_btn.on_click(on_clear_dash)
export_btn.on_click(on_export_dash)

# Build layout
config_box = VBox([
    dataset_header,
    n_samples_slider,
    noise_slider,
    model_header,
    test_size_slider,
    model_type
])

button_box = HBox([generate_btn_dash, clear_btn, export_btn])

output_tabs = widgets.Tab(children=[output_data, output_viz, output_metrics])
output_tabs.set_title(0, '📊 Data')
output_tabs.set_title(1, '📈 Visualization')
output_tabs.set_title(2, '📋 Metrics')

# Combine everything
dashboard = VBox([
    title_label,
    widgets.HTML('<hr style="margin: 20px 0;">'),
    HBox([
        VBox([widgets.HTML('<b>Configuration</b>'), config_box], layout=widgets.Layout(width='40%')),
        VBox([widgets.HTML('<b>Actions</b>'), button_box], layout=widgets.Layout(width='50%', padding='10px'))
    ]),
    status_output,
    widgets.HTML('<hr style="margin: 20px 0;">'),
    output_tabs
])

display(dashboard)

## 6. Create a Complete Dashboard

Combine all elements into a comprehensive, professional dashboard with multiple features.

In [ ]:
# Create tabs for organized output
output_text = widgets.Output()
output_plot = widgets.Output()
output_stats = widgets.Output()

tab = widgets.Tab(children=[output_text, output_plot, output_stats])
tab.set_title(0, 'Text Output')
tab.set_title(1, 'Plots')
tab.set_title(2, 'Statistics')

# Function to generate sample data and display results
def generate_data(n_samples=100, distribution='normal'):
    # Generate data based on selected distribution
    if distribution == 'normal':
        data = np.random.normal(loc=0, scale=1, size=n_samples)
    elif distribution == 'uniform':
        data = np.random.uniform(low=-3, high=3, size=n_samples)
    else:  # exponential
        data = np.random.exponential(scale=1, size=n_samples)
    
    # Clear and update text output
    with output_text:
        clear_output()
        print(f'Generated {n_samples} samples from {distribution} distribution')
        print(f'Data summary:')
        print(f'  Mean: {data.mean():.4f}')
        print(f'  Std Dev: {data.std():.4f}')
        print(f'  Min: {data.min():.4f}')
        print(f'  Max: {data.max():.4f}')
    
    # Clear and update plot
    with output_plot:
        clear_output()
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        ax1.hist(data, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
        ax1.set_title('Histogram')
        ax1.set_xlabel('Value')
        ax1.set_ylabel('Frequency')
        
        ax2.boxplot(data, vert=True)
        ax2.set_ylabel('Value')
        ax2.set_title('Box Plot')
        
        plt.tight_layout()
        plt.show()
    
    # Clear and update statistics
    with output_stats:
        clear_output()
        stats_df = pd.DataFrame({
            'Statistic': ['Count', 'Mean', 'Std Dev', 'Min', '25%', '50%', '75%', 'Max'],
            'Value': [len(data), data.mean(), data.std(), data.min(), 
                     np.percentile(data, 25), np.percentile(data, 50), 
                     np.percentile(data, 75), data.max()]
        })
        display(stats_df.to_html(index=False), raw=True)

# Create controls
samples_control = widgets.IntSlider(value=100, min=10, max=1000, step=50, description='Samples:')
dist_control = widgets.Dropdown(options=['normal', 'uniform', 'exponential'], 
                               value='normal', description='Distribution:')
generate_btn = widgets.Button(description='Generate Data', button_style='success')

def on_generate_click(b):
    generate_data(samples_control.value, dist_control.value)

generate_btn.on_click(on_generate_click)

# Display UI
display(widgets.HTML('<b>Data Generator:</b>'))
display(HBox([samples_control, dist_control, generate_btn]))
display(tab)

## 5. Display Output with Widgets

Use Output widgets to capture and display results from widget-triggered functions.

In [ ]:
# Example 3: Button click handler
click_count = {'count': 0}
button_output = widgets.Output()
counter_display = widgets.HTML(value='<b>Clicks: 0</b>')

def on_button_click(b):
    click_count['count'] += 1
    counter_display.value = f'<b style="color: #27AE60;">Clicks: {click_count["count"]}</b>'

click_button = widgets.Button(description='Click Me!', button_style='info')
click_button.on_click(on_button_click)

display(HBox([click_button, counter_display]))

In [ ]:
# Example 2: Using interact() for automatic widget generation
def celsius_to_fahrenheit(celsius=0):
    fahrenheit = (celsius * 9/5) + 32
    print(f'{celsius}°C = {fahrenheit:.1f}°F')

display(widgets.HTML('<b>Temperature Converter (interact):</b>'))
interact(celsius_to_fahrenheit, celsius=widgets.FloatSlider(min=-50, max=50, step=0.1))

In [ ]:
# Example 1: Using observe() to respond to widget changes
counter_output = widgets.Output()

def increment_counter(count):
    with counter_output:
        clear_output()
        print(f'Count: {count}')
        print(f'Double: {count * 2}')
        print(f'Square: {count ** 2}')

counter_slider = widgets.IntSlider(value=0, min=0, max=20, step=1, description='Count:')
counter_slider.observe(lambda change: increment_counter(change['new']), names='value')

display(widgets.HTML('<b>Counter Calculator:</b>'))
display(counter_slider, counter_output)

## 4. Connect Widgets to Functions

Use `observe()` and `interact()` to bind widget events to Python functions for dynamic behavior.

In [ ]:
# Create a control panel using VBox and HBox
title = widgets.HTML('<h3 style="color: #2E86C1;">Data Analysis Control Panel</h3>')

# Row 1: File upload and format selection
file_upload = widgets.FileUpload(accept='.csv', multiple=False)
file_format = widgets.RadioButtons(options=['CSV', 'JSON'], description='Format:')
row1 = HBox([file_upload, widgets.Label(width='20px'), file_format])

# Row 2: Sample size and random seed
sample_size = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Sample Size:')
random_seed = widgets.IntText(value=42, description='Random Seed:')
row2 = HBox([sample_size, random_seed])

# Row 3: Analysis type selection
analysis_type = widgets.SelectMultiple(
    options=['Statistics', 'Visualization', 'Correlation', 'Distribution'],
    description='Analysis:',
    rows=4
)

# Buttons
load_btn = widgets.Button(description='Load Data', button_style='success', icon='upload')
reset_btn = widgets.Button(description='Reset', button_style='warning', icon='refresh')
button_row = HBox([load_btn, reset_btn])

# Combine all rows
control_panel = VBox([
    title,
    widgets.HTML('<hr>'),
    widgets.HTML('<b>Data Loading:</b>'),
    row1,
    widgets.HTML('<b>Parameters:</b>'),
    row2,
    widgets.HTML('<b>Analysis Options:</b>'),
    analysis_type,
    button_row
])

display(control_panel)

## 3. Build Interactive Controls

Combine multiple widgets using VBox and HBox containers for organized layouts.

In [ ]:
# Text Input Widget
text_input = widgets.Text(
    value='Enter text here',
    placeholder='Type something',
    description='Name:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

# Dropdown Widget
dropdown = widgets.Dropdown(
    options=['Option 1', 'Option 2', 'Option 3'],
    value='Option 1',
    description='Choose:',
    style={'description_width': '80px'}
)

# Checkbox Widget
checkbox = widgets.Checkbox(
    value=False,
    description='Enable Feature'
)

display(HBox([widgets.Label('Text Input:'), text_input]))
display(HBox([widgets.Label('Dropdown:'), dropdown]))
display(HBox([widgets.Label('Checkbox:'), checkbox]))

In [ ]:
# Slider Widget
slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Select Value:',
    orientation='horizontal',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

slider_output = widgets.Output()
display(slider, slider_output)

@slider.observe
def handle_slider_change(change):
    with slider_output:
        clear_output()
        print(f'Current Value: {change["new"]}')

In [ ]:
# Button Widget
button = widgets.Button(
    description='Click Me!',
    button_style='info',  # 'success', 'info', 'warning', 'danger', ''
    tooltip='Click to trigger action',
    icon='check'  # FontAwesome icon
)
display(button)

## 2. Create Basic Widgets

Learn how to create individual widgets: buttons, sliders, text inputs, and dropdowns.

In [ ]:
# Import core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import Jupyter widgets
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Output, interactive, interact
from IPython.display import display, HTML, clear_output

# Set display options
%matplotlib inline
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Import Required Libraries

Import libraries for GUI creation, visualization, and data handling.

# Jupyter Interactive GUI Dashboard

This notebook demonstrates how to build interactive graphical user interfaces (GUIs) using Jupyter widgets (`ipywidgets`). You'll learn to create buttons, sliders, dropdowns, and complete dashboards with real-time updates.